# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [7]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler, Normalizer, StandardScaler, MinMaxScaler
from pyspark.sql.functions import udf, sum as Fsum, pow as Fpow, col, sqrt as Fsqrt, concat, lit
from pyspark.sql.functions import desc, asc
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
!pip install ua-parser
from ua_parser import user_agent_parser
import datetime

import numpy as np
import pandas as pd
import re
%matplotlib inline
import matplotlib.pyplot as plt

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
ua_string = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36'
parsed_string = user_agent_parser.Parse(ua_string)
parsed_string['user_agent']['family']
parsed_string['os']['family']

In [9]:
parsed_string['user_agent']['family']

'Chrome'

In [10]:
parsed_string['os']['family']

'Mac OS X'

In [11]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkcapstoneproject") \
    .getOrCreate()


# Load and Clean Dataset
Clean your dataset, checking for invalid or missing data. For example, records without userids or sessionids. In this workspace, the filename is `mini_sparkify_event_data.json`.

In [109]:
mini_sparkify = 'mini_sparkify_event_data.json'

In [110]:
df = spark.read.json(mini_sparkify)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [5]:
df.show(1)

+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|        artist|     auth|firstName|gender|itemInSession|lastName|   length|level|       location|method|    page| registration|sessionId|     song|status|           ts|           userAgent|userId|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|Bakersfield, CA|   PUT|NextSong|1538173362000|       29|Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
only showi

In [6]:
df.take(2)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30'),
 Row(artist='Five Iron Frenzy', auth='Logged In', firstName='Micah', gender='M', itemInSession=79, lastName='Long', length=236.09424, level='free', location='Boston-Cambridge-Newton, MA-NH', method='PUT', page='NextSong', registration=1538331630000, sessionId=8, song='Canada', status=200, ts=1538352180000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.103 Safari/537.36"', userId='9')]

In [7]:
df.count()

286500

In [8]:
df.describe("sessionId").show()

+-------+-----------------+
|summary|        sessionId|
+-------+-----------------+
|  count|           286500|
|   mean|1041.526554973822|
| stddev|726.7762634630741|
|    min|                1|
|    max|             2474|
+-------+-----------------+



In [9]:
df.describe("userId").show()

+-------+-----------------+
|summary|           userId|
+-------+-----------------+
|  count|           286500|
|   mean|59682.02278593872|
| stddev|109091.9499991047|
|    min|                 |
|    max|               99|
+-------+-----------------+



In [111]:
df_valid = df.dropna(how = "any", subset = ["userId", "sessionId"])

In [11]:
df_valid.count()

286500

In [12]:
df_valid.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



In [13]:
df_valid.select("sessionId").dropDuplicates().sort("sessionId").show()

+---------+
|sessionId|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       15|
|       16|
|       17|
|       18|
|       19|
|       20|
|       21|
+---------+
only showing top 20 rows



In [38]:
df_valid.select("sessionId").dropDuplicates().sort("sessionId").count()

2312

In [112]:
df_valid = df_valid.filter((df_valid.userId != "") | (df_valid.sessionId != ""))

In [19]:
df_valid.count()

278154

In [16]:
df_valid.select("gender").dropDuplicates().sort("gender").show()

+------+
|gender|
+------+
|     F|
|     M|
+------+



In [22]:
#total unique users
df_valid.select(["userId"]).dropDuplicates().count()

225

In [17]:
#users by gender
df_valid.select(["userId", "gender"]).dropDuplicates().groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  104|
|     M|  121|
+------+-----+



In [19]:
# some users have more than one method
df_valid.select("userId", "method").dropDuplicates().groupBy("method").count().show()

+------+-----+
|method|count|
+------+-----+
|   PUT|  225|
|   GET|  224|
+------+-----+



In [23]:
# some users have more than one level
df_valid.select("userId", "level").dropDuplicates().groupBy("level").count().show()

+-----+-----+
|level|count|
+-----+-----+
| free|  195|
| paid|  165|
+-----+-----+



In [23]:
# 114 unique location
df_valid.select("location").dropDuplicates().sort("location").count()

114

In [29]:
df_valid.groupBy("location").count().sort(desc("count")).show()

+--------------------+-----+
|            location|count|
+--------------------+-----+
|Los Angeles-Long ...|30131|
|New York-Newark-J...|23684|
|Boston-Cambridge-...|13873|
|Houston-The Woodl...| 9499|
|Charlotte-Concord...| 7780|
|Dallas-Fort Worth...| 7605|
|Louisville/Jeffer...| 6880|
|Philadelphia-Camd...| 5890|
|Chicago-Napervill...| 5114|
|    St. Louis, MO-IL| 4858|
|Phoenix-Mesa-Scot...| 4846|
|Vineland-Bridgeto...| 4825|
|          Wilson, NC| 4659|
|Denver-Aurora-Lak...| 4453|
|           Ionia, MI| 4428|
|San Antonio-New B...| 4373|
|        Danville, VA| 4257|
|Atlanta-Sandy Spr...| 4236|
|New Haven-Milford...| 4007|
|         Jackson, MS| 3839|
+--------------------+-----+
only showing top 20 rows



In [27]:
df_valid.select("location").dropDuplicates().describe("location").show()

+-------+-----------------+
|summary|         location|
+-------+-----------------+
|  count|              114|
|   mean|             null|
| stddev|             null|
|    min|       Albany, OR|
|    max|Winston-Salem, NC|
+-------+-----------------+



In [28]:
df_valid.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|              Logout|
|            NextSong|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



In [30]:
df_valid.select("userAgent").dropDuplicates().sort("userAgent").count()

56

In [32]:
df_valid.select("status").dropDuplicates().sort("status").show()

+------+
|status|
+------+
|   200|
|   307|
|   404|
+------+



In [33]:
df_valid.select("auth").dropDuplicates().sort("auth").show()

+---------+
|     auth|
+---------+
|Cancelled|
|Logged In|
+---------+



In [35]:
df_valid.select("itemInSession").dropDuplicates().sort("itemInSession").count()

1311

In [36]:
df_valid.describe("length").show()

+-------+-----------------+
|summary|           length|
+-------+-----------------+
|  count|           228108|
|   mean|249.1171819778458|
| stddev|99.23517921058361|
|    min|          0.78322|
|    max|       3024.66567|
+-------+-----------------+



In [37]:
df_valid.describe("registration").show()

+-------+--------------------+
|summary|        registration|
+-------+--------------------+
|  count|              278154|
|   mean|1.535358834084427...|
| stddev| 3.291321616327586E9|
|    min|       1521380675000|
|    max|       1543247354000|
+-------+--------------------+



In [39]:
df_valid.describe("artist").show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|            228108|
|   mean| 551.0852017937219|
| stddev|1217.7693079161374|
|    min|               !!!|
|    max| ÃÂlafur Arnalds|
+-------+------------------+



In [41]:
df_valid.select("artist").dropDuplicates().sort("artist").count()

17656

In [40]:
df_valid.describe("song").show()

+-------+--------------------+
|summary|                song|
+-------+--------------------+
|  count|              228108|
|   mean|            Infinity|
| stddev|                 NaN|
|    min|ÃÂg ÃÂtti Gr...|
|    max|ÃÂau hafa slopp...|
+-------+--------------------+



In [42]:
df_valid.select("song").dropDuplicates().sort("song").count()

58481

In [68]:
df_valid.select(["userId", "page"]).filter(df_valid.page == "Cancellation Confirmation").groupby("userId").count().count()

52

In [70]:
df_valid.filter(df_valid.page == "Cancellation Confirmation").take(5)

[Row(artist=None, auth='Cancelled', firstName='Adriel', gender='M', itemInSession=104, lastName='Mendoza', length=None, level='paid', location='Kansas City, MO-KS', method='GET', page='Cancellation Confirmation', registration=1535623466000, sessionId=514, song=None, status=200, ts=1538943990000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.77.4 (KHTML, like Gecko) Version/7.0.5 Safari/537.77.4"', userId='18'),
 Row(artist=None, auth='Cancelled', firstName='Diego', gender='M', itemInSession=56, lastName='Mckee', length=None, level='paid', location='Phoenix-Mesa-Scottsdale, AZ', method='GET', page='Cancellation Confirmation', registration=1537167593000, sessionId=540, song=None, status=200, ts=1539033046000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='32'),
 Row(artist=None, auth='Cancelled', firstName='Mason', gender='M', itemInSession=10, la

In [53]:
df_valid.select(["artist", "page", "song", "level", "length","status"]).filter(df_valid.userId == "32").collect()

[Row(artist='RJD2', page='NextSong', song='Beyond The Beyond (Instrumental)', level='free', length=237.53098, status=200),
 Row(artist="Guns N' Roses", page='NextSong', song='Paradise City', level='free', length=406.17751, status=200),
 Row(artist='Lady GaGa', page='NextSong', song='Alejandro', level='free', length=274.18077, status=200),
 Row(artist=None, page='Roll Advert', song=None, level='free', length=None, status=200),
 Row(artist='LL Cool J', page='NextSong', song='Luv U Better', level='free', length=242.6771, status=200),
 Row(artist=None, page='Thumbs Up', song=None, level='free', length=None, status=307),
 Row(artist='Kelly Clarkson', page='NextSong', song='My Life Would Suck Without You', level='free', length=211.56526, status=200),
 Row(artist=None, page='Add to Playlist', song=None, level='free', length=None, status=200),
 Row(artist='Natiruts', page='NextSong', song='Jamaica Roots II(Agora E Sempre)', level='free', length=210.88608, status=200),
 Row(artist='Alan Barratt

In [72]:
df_valid.select(["page", "level", "length", "ts"]).filter(df_valid.userId == "18").collect()

[Row(page='Home', level='paid', length=None, ts=1538499917000),
 Row(page='NextSong', level='paid', length=275.12118, ts=1538499933000),
 Row(page='NextSong', level='paid', length=268.32934, ts=1538500208000),
 Row(page='NextSong', level='paid', length=178.72934, ts=1538500476000),
 Row(page='NextSong', level='paid', length=188.76036, ts=1538500654000),
 Row(page='NextSong', level='paid', length=167.91465, ts=1538500842000),
 Row(page='Settings', level='paid', length=None, ts=1538500856000),
 Row(page='NextSong', level='paid', length=331.38893, ts=1538501009000),
 Row(page='NextSong', level='paid', length=247.69261, ts=1538501340000),
 Row(page='NextSong', level='paid', length=153.52118, ts=1538501587000),
 Row(page='NextSong', level='paid', length=244.13995, ts=1538501740000),
 Row(page='NextSong', level='paid', length=140.64281, ts=1538501984000),
 Row(page='Settings', level='paid', length=None, ts=1538502021000),
 Row(page='NextSong', level='paid', length=232.56771, ts=1538502124000

In [89]:
df_valid.select(["userId", "page", "level", "length", "ts"]).groupby("userId").agg(Fsum("length")/(60*60)).collect()

[Row(userId='100010', (sum(length) / 3600)=18.59469370833334),
 Row(userId='200002', (sum(length) / 3600)=26.113576649999978),
 Row(userId='125', (sum(length) / 3600)=0.5803091944444446),
 Row(userId='51', (sum(length) / 3600)=145.3544007777779),
 Row(userId='124', (sum(length) / 3600)=281.19780355277777),
 Row(userId='7', (sum(length) / 3600)=10.56502419444445),
 Row(userId='54', (sum(length) / 3600)=197.59581098333362),
 Row(userId='15', (sum(length) / 3600)=132.5854460583334),
 Row(userId='155', (sum(length) / 3600)=55.21646997222223),
 Row(userId='132', (sum(length) / 3600)=134.19969551111103),
 Row(userId='154', (sum(length) / 3600)=5.738895530555557),
 Row(userId='100014', (sum(length) / 3600)=18.806520022222234),
 Row(userId='101', (sum(length) / 3600)=124.29555963055525),
 Row(userId='11', (sum(length) / 3600)=44.35276751111106),
 Row(userId='138', (sum(length) / 3600)=142.34718965555524),
 Row(userId='300017', (sum(length) / 3600)=249.2797167250004),
 Row(userId='29', (sum(len

In [69]:
df_valid.select(["page", "level", "length"]).filter(df_valid.userId == "125").collect()

[Row(page='NextSong', level='free', length=337.91955),
 Row(page='NextSong', level='free', length=230.03383),
 Row(page='NextSong', level='free', length=203.2322),
 Row(page='NextSong', level='free', length=210.83383),
 Row(page='NextSong', level='free', length=391.26159),
 Row(page='NextSong', level='free', length=213.60281),
 Row(page='NextSong', level='free', length=178.41587),
 Row(page='NextSong', level='free', length=323.81342),
 Row(page='Roll Advert', level='free', length=None),
 Row(page='Cancel', level='free', length=None),
 Row(page='Cancellation Confirmation', level='free', length=None)]

In [71]:
df_valid.select(["page", "level", "length"]).filter(df_valid.userId == "105").collect()

[Row(page='Home', level='free', length=None),
 Row(page='NextSong', level='free', length=257.59302),
 Row(page='NextSong', level='free', length=162.45506),
 Row(page='NextSong', level='free', length=242.36363),
 Row(page='NextSong', level='free', length=221.49179),
 Row(page='NextSong', level='free', length=221.85751),
 Row(page='NextSong', level='free', length=481.48853),
 Row(page='NextSong', level='free', length=267.49342),
 Row(page='NextSong', level='free', length=216.76363),
 Row(page='NextSong', level='free', length=348.57751),
 Row(page='Home', level='free', length=None),
 Row(page='NextSong', level='free', length=179.40853),
 Row(page='NextSong', level='free', length=486.05995),
 Row(page='NextSong', level='free', length=276.47955),
 Row(page='NextSong', level='free', length=488.64608),
 Row(page='NextSong', level='free', length=215.97995),
 Row(page='NextSong', level='free', length=321.54077),
 Row(page='Add to Playlist', level='free', length=None),
 Row(page='NextSong', leve

In [78]:
df_valid.select(["page", "userAgent"]).filter(df_valid.userId == "17").collect()

[Row(page='Home', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='Roll Advert', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong', userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'),
 Row(page='NextSong'

In [76]:
df_valid.select(["page", "userAgent"]).filter(df_valid.userId == "125").collect()

[Row(page='NextSong', userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(page='NextSong', userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(page='NextSong', userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(page='NextSong', userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(page='NextSong', userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(page='NextSong', userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'),
 Row(page='NextSong', userAgent='"Mozilla/5.0 

In [90]:
# df_valid.select(["page", "userAgent"]).filter(df_valid.userId == "105").collect()

In [13]:
df_valid.select(["userId", "ts", "page"]).filter(df_valid.page == "NextSong").groupby("userId").count().orderBy("userId").show()

+------+-----+
|userId|count|
+------+-----+
|    10|  673|
|   100| 2682|
|100001|  133|
|100002|  195|
|100003|   51|
|100004|  942|
|100005|  154|
|100006|   26|
|100007|  423|
|100008|  772|
|100009|  518|
|100010|  275|
|100011|   11|
|100012|  476|
|100013| 1131|
|100014|  257|
|100015|  800|
|100016|  530|
|100017|   52|
|100018| 1002|
+------+-----+
only showing top 20 rows



In [14]:
df_valid.select(["userId", "ts", "page"]).filter(df_valid.page == "Downgrade").groupby("userId").count().orderBy("userId").show()

+------+-----+
|userId|count|
+------+-----+
|    10|    7|
|   100|   30|
|100002|    2|
|100004|   10|
|100007|    7|
|100008|   10|
|100009|    5|
|100012|    4|
|100013|   13|
|100014|    3|
|100015|    5|
|100016|    6|
|100018|    9|
|100023|    9|
|100025|    5|
|   101|   22|
|   103|   13|
|   104|   24|
|   105|   10|
|   106|    3|
+------+-----+
only showing top 20 rows



In [31]:
df_valid.select(["userId", "ts", "page"]).filter(df_valid.page == "Cancellation Confirmation").sort("userId").show(60) #.groupby("userId").count().orderBy(desc("count")).show(70)

+------+-------------+--------------------+
|userId|           ts|                page|
+------+-------------+--------------------+
|100001|1538498205000|Cancellation Conf...|
|100003|1539274781000|Cancellation Conf...|
|100005|1539971825000|Cancellation Conf...|
|100006|1538753070000|Cancellation Conf...|
|100007|1543491909000|Cancellation Conf...|
|100009|1540611104000|Cancellation Conf...|
|100011|1538417085000|Cancellation Conf...|
|100012|1541100900000|Cancellation Conf...|
|100013|1541184816000|Cancellation Conf...|
|100014|1542740649000|Cancellation Conf...|
|100015|1543073753000|Cancellation Conf...|
|100017|1540062847000|Cancellation Conf...|
|100019|1538953071000|Cancellation Conf...|
|100021|1543143782000|Cancellation Conf...|
|100022|1542038645000|Cancellation Conf...|
|100023|1539475237000|Cancellation Conf...|
|100024|1538695038000|Cancellation Conf...|
|100025|1540299152000|Cancellation Conf...|
|   101|1539729037000|Cancellation Conf...|
|   103|1541340091000|Cancellati

In [20]:
df_valid.select("location").show()

+--------------------+
|            location|
+--------------------+
|     Bakersfield, CA|
|Boston-Cambridge-...|
|     Bakersfield, CA|
|Boston-Cambridge-...|
|     Bakersfield, CA|
|Boston-Cambridge-...|
|Boston-Cambridge-...|
|     Bakersfield, CA|
|     Bakersfield, CA|
|     Bakersfield, CA|
|Boston-Cambridge-...|
|Boston-Cambridge-...|
|     Bakersfield, CA|
|Boston-Cambridge-...|
|Boston-Cambridge-...|
|     Bakersfield, CA|
|Boston-Cambridge-...|
|     Tallahassee, FL|
|     Bakersfield, CA|
|Boston-Cambridge-...|
+--------------------+
only showing top 20 rows



In [45]:
state = udf(lambda x: x.split(" ")[-1][-2:], StringType())

In [46]:
df_test = df_valid
df_test = df_test.withColumn("state", state("location"))

In [51]:
df_test.select("state").groupby("state").count().show(40)

+-----+-----+
|state|count|
+-----+-----+
|   AZ| 4846|
|   SC| 8448|
|   LA| 1171|
|   NJ| 7816|
|   OR|   23|
|   VA| 6514|
|   KY| 3462|
|   NH|18637|
|   MI| 7216|
|   NV| 2042|
|   WI| 8697|
|   ID|  317|
|   CA|46771|
|   CT| 6720|
|   MT| 2386|
|   NC|11771|
|   MD| 8600|
|   MO| 1003|
|   IL| 6960|
|   WA| 3891|
|   MS| 4634|
|   AL| 5573|
|   IN| 8230|
|   OH| 3432|
|   TN| 1672|
|   IA|  838|
|   PA|26607|
|   NY| 4536|
|   TX|23494|
|   WV| 6350|
|   GA| 4236|
|   MA|  927|
|   KS| 2562|
|   FL|13190|
|   CO| 7493|
|   AK| 3563|
|   AR|  582|
|   OK| 1842|
|   UT| 1102|
+-----+-----+



225 unique user, 2 gender F/M, 2 level free/paid, 2 method GET/PUT, 114 location, 2312 sessionId, 2 auth cancelled/loggedIn, 3 status 200/307/404, 1311 Iteminsession, 56 userAgent (software, OS), 19 page, 17656 artist, 58481 song, ts continous, length continous

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [40]:
cancel_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

In [42]:
next_event = udf(lambda x: 1 if x == "NextSong" else 0, IntegerType())

In [43]:
from pyspark.sql import Window
df_valid = df_valid.withColumn("cancel_", cancel_event("page"))

In [44]:
df_valid = df_valid.withColumn("next_", next_event("page"))

In [45]:
df_agg = df_valid.select(["userId", "cancel_","next_"]).groupby("userId").sum().sort("userId")

In [46]:
df_agg.show()

+------+------------+----------+
|userId|sum(cancel_)|sum(next_)|
+------+------------+----------+
|    10|           0|       673|
|   100|           0|      2682|
|100001|           1|       133|
|100002|           0|       195|
|100003|           1|        51|
|100004|           0|       942|
|100005|           1|       154|
|100006|           1|        26|
|100007|           1|       423|
|100008|           0|       772|
|100009|           1|       518|
|100010|           0|       275|
|100011|           1|        11|
|100012|           1|       476|
|100013|           1|      1131|
|100014|           1|       257|
|100015|           1|       800|
|100016|           0|       530|
|100017|           1|        52|
|100018|           0|      1002|
+------+------------+----------+
only showing top 20 rows



In [35]:
df_agg.show()

+------+------------+
|userId|sum(cancel_)|
+------+------------+
|      |           0|
|    10|           0|
|   100|           0|
|100001|           1|
|100002|           0|
|100003|           1|
|100004|           0|
|100005|           1|
|100006|           1|
|100007|           1|
|100008|           0|
|100009|           1|
|100010|           0|
|100011|           1|
|100012|           1|
|100013|           1|
|100014|           1|
|100015|           1|
|100016|           0|
|100017|           1|
+------+------------+
only showing top 20 rows



In [47]:
df_valid.select("page").dropDuplicates().sort("page").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|              Logout|
|            NextSong|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



In [113]:
col_names_0_1 = ["About", "Add Friend", "Add to Playlist", "Cancellation Confirmation",\
             "Downgrade", "Error", "Help", "Home", "Logout", "NextSong", "Roll Advert",\
             "Settings", "Submit Downgrade", "Submit Upgrade", "Thumbs Down", "Thumbs Up", "Upgrade"]

def make_category_cols(df_valid, col_names_0_1):
    
    event = udf(lambda x: 1 if x == "name" else 0, IntegerType())
    
    col_name_new_list = ["userId"]
    for name in col_names_0_1:
        
        event = udf(lambda x: 1 if x == str(name) else 0, IntegerType())
        col_name_new = str(name)+"_"
        col_name_new_list.append(col_name_new)
        df_valid = df_valid.withColumn(col_name_new, event("page"))
        
    event_user_agent = udf(lambda x: user_agent_parser.Parse(x)['user_agent']['family'], StringType())
    event_os = udf(lambda x: user_agent_parser.Parse(x)['os']['family'], StringType())
    event_state = udf(lambda x: x.split(" ")[-1][-2:], StringType())
    
    df_valid = df_valid.withColumn("browser", event_user_agent("userAgent"))
    df_valid = df_valid.withColumn("os", event_os("userAgent"))
    df_valid = df_valid.withColumn("state", event_state("location"))
    
    return df_valid, col_name_new_list  
        

In [118]:
df_valid, col_name_new_list = make_category_cols(df_valid, col_names_0_1)

In [67]:
df_valid.select(["browser", "os", "state"]).show()

+-------+--------+-----+
|browser|      os|state|
+-------+--------+-----+
|Firefox| Windows|   CA|
| Chrome| Windows|   NH|
|Firefox| Windows|   CA|
| Chrome| Windows|   NH|
|Firefox| Windows|   CA|
| Chrome| Windows|   NH|
| Chrome| Windows|   NH|
|Firefox| Windows|   CA|
|Firefox| Windows|   CA|
|Firefox| Windows|   CA|
| Chrome| Windows|   NH|
| Chrome| Windows|   NH|
|Firefox| Windows|   CA|
| Chrome| Windows|   NH|
| Chrome| Windows|   NH|
|Firefox| Windows|   CA|
| Chrome| Windows|   NH|
| Chrome|Mac OS X|   FL|
|Firefox| Windows|   CA|
| Chrome| Windows|   NH|
+-------+--------+-----+
only showing top 20 rows



In [124]:
df_valid.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', About_=0, Add Friend_=0, Add to Playlist_=0, Cancellation Confirmation_=0, Downgrade_=0, Error_=0, Help_=0, Home_=0, Logout_=0, NextSong_=1, Roll Advert_=0, Settings_=0, Submit Downgrade_=0, Submit Upgrade_=0, Thumbs Down_=0, Thumbs Up_=0, Upgrade_=0, browser='Firefox', os='Windows', state='CA')]

In [125]:
df_valid.select(col_name_new_list).take(5)

[Row(userId='30', About_=0, Add Friend_=0, Add to Playlist_=0, Cancellation Confirmation_=0, Downgrade_=0, Error_=0, Help_=0, Home_=0, Logout_=0, NextSong_=1, Roll Advert_=0, Settings_=0, Submit Downgrade_=0, Submit Upgrade_=0, Thumbs Down_=0, Thumbs Up_=0, Upgrade_=0),
 Row(userId='9', About_=0, Add Friend_=0, Add to Playlist_=0, Cancellation Confirmation_=0, Downgrade_=0, Error_=0, Help_=0, Home_=0, Logout_=0, NextSong_=1, Roll Advert_=0, Settings_=0, Submit Downgrade_=0, Submit Upgrade_=0, Thumbs Down_=0, Thumbs Up_=0, Upgrade_=0),
 Row(userId='30', About_=0, Add Friend_=0, Add to Playlist_=0, Cancellation Confirmation_=0, Downgrade_=0, Error_=0, Help_=0, Home_=0, Logout_=0, NextSong_=1, Roll Advert_=0, Settings_=0, Submit Downgrade_=0, Submit Upgrade_=0, Thumbs Down_=0, Thumbs Up_=0, Upgrade_=0),
 Row(userId='9', About_=0, Add Friend_=0, Add to Playlist_=0, Cancellation Confirmation_=0, Downgrade_=0, Error_=0, Help_=0, Home_=0, Logout_=0, NextSong_=1, Roll Advert_=0, Settings_=0, S

In [119]:
print(col_name_new_list)

['userId', 'About_', 'Add Friend_', 'Add to Playlist_', 'Cancellation Confirmation_', 'Downgrade_', 'Error_', 'Help_', 'Home_', 'Logout_', 'NextSong_', 'Roll Advert_', 'Settings_', 'Submit Downgrade_', 'Submit Upgrade_', 'Thumbs Down_', 'Thumbs Up_', 'Upgrade_']


In [126]:
# def make_agg(df_valid):
#     col_names_0_1
    # sum
# sum of all numercial 0/1    
df_agg_1 = df_valid.select(col_name_new_list).groupby("userId").sum().sort("userId")
# sum of continous length song (need to divide by 3600 to get hour)
df_agg_2 = df_valid.select(["userId", "length"]).filter(df_valid.page == "NextSong").groupby("userId").sum().sort("userId")

In [127]:
df_agg_1.take(1)

[Row(userId='10', sum(About_)=2, sum(Add Friend_)=12, sum(Add to Playlist_)=9, sum(Cancellation Confirmation_)=0, sum(Downgrade_)=7, sum(Error_)=0, sum(Help_)=1, sum(Home_)=30, sum(Logout_)=11, sum(NextSong_)=673, sum(Roll Advert_)=1, sum(Settings_)=7, sum(Submit Downgrade_)=0, sum(Submit Upgrade_)=0, sum(Thumbs Down_)=4, sum(Thumbs Up_)=37, sum(Upgrade_)=0)]

In [128]:
col_list_last = ["userId", "gender", "state", "ts", "os", "browser"]
# last time stamp categories
df_agg_3 = df_valid.select(col_list_last).groupby(["userId", "gender", "state", "os", "browser"]).max("ts").sort("userId")
# first time stamp
df_agg_4 = df_valid.select(col_list_last).groupby(["userId"]).min("ts").sort("userId")

In [129]:
df_agg = df_agg_1.join(df_agg_2, ["UserId"]).join(df_agg_3, ["UserId"]).join(df_agg_4, ["UserId"])

In [131]:
from functools import reduce
old_columns = df_agg.schema.names
print(old_columns)


['userId', 'sum(About_)', 'sum(Add Friend_)', 'sum(Add to Playlist_)', 'sum(Cancellation Confirmation_)', 'sum(Downgrade_)', 'sum(Error_)', 'sum(Help_)', 'sum(Home_)', 'sum(Logout_)', 'sum(NextSong_)', 'sum(Roll Advert_)', 'sum(Settings_)', 'sum(Submit Downgrade_)', 'sum(Submit Upgrade_)', 'sum(Thumbs Down_)', 'sum(Thumbs Up_)', 'sum(Upgrade_)', 'sum(length)', 'gender', 'state', 'os', 'browser', 'max(ts)', 'min(ts)']


In [132]:
new_columns = ['userId', 'About', 'Friend', 'Playlist', 'Churn', 'Downgrade', 'Error',\
               'Help', 'Home', 'Logout', 'Song', 'Advert', 'Settings', 'Submitdown', \
               'Submitup', 'Thumbsdown', 'Thumbsup', 'Upgrade', 'listen_length', \
               'gender', 'state', 'os', 'browser', 'max_ts', 'min_ts']
df_agg = reduce(lambda df_agg, idx: df_agg.withColumnRenamed(old_columns[idx], new_columns[idx]), range(len(old_columns)), df_agg)

In [133]:
df_agg.take(5)

[Row(userId='100010', About=1, Friend=4, Playlist=7, Churn=0, Downgrade=0, Error=0, Help=2, Home=11, Logout=5, Song=275, Advert=52, Settings=0, Submitdown=0, Submitup=0, Thumbsdown=5, Thumbsup=17, Upgrade=2, listen_length=66940.89735000003, gender='F', state='CT', os='iOS', browser='Mobile Safari', max_ts=1542823952000, min_ts=1539003534000),
 Row(userId='200002', About=3, Friend=4, Playlist=8, Churn=0, Downgrade=5, Error=0, Help=2, Home=20, Logout=5, Song=387, Advert=7, Settings=3, Submitdown=0, Submitup=1, Thumbsdown=6, Thumbsup=21, Upgrade=2, listen_length=94008.87593999993, gender='M', state='WI', os='iOS', browser='Mobile Safari', max_ts=1542324354000, min_ts=1538393430000),
 Row(userId='125', About=0, Friend=0, Playlist=0, Churn=1, Downgrade=0, Error=0, Help=0, Home=0, Logout=0, Song=8, Advert=1, Settings=0, Submitdown=0, Submitup=0, Thumbsdown=0, Thumbsup=0, Upgrade=0, listen_length=2089.1131000000005, gender='M', state='TX', os='Mac OS X', browser='Chrome', max_ts=1539318918000

In [143]:
def make_agg_join_df(df_valid, col_name_new_list):
    
    # sum of all numercial 0/1    
    df_agg_1 = df_valid.select(col_name_new_list).groupby("userId").sum().sort("userId")
    # sum of continous length song (need to divide by 3600 to get hour)
    df_agg_2 = df_valid.select(["userId", "length"]).filter(df_valid.page == "NextSong").groupby("userId").sum().sort("userId")
    col_list_last = ["userId", "gender", "state", "ts", "os", "browser"]
    # last time stamp categories
    df_agg_3 = df_valid.select(col_list_last).groupby(["userId", "gender", "state", "os", "browser"]).max("ts").sort("userId")
    # first time stamp
    df_agg_4 = df_valid.select(col_list_last).groupby(["userId"]).min("ts").sort("userId")    
    df_agg = df_agg_1.join(df_agg_2, ["UserId"]).join(df_agg_3, ["UserId"]).join(df_agg_4, ["UserId"])
    old_columns = df_agg.schema.names
    new_columns = ['userId', 'About', 'Friend', 'Playlist', 'Churn', 'Downgrade', 'Error',\
               'Help', 'Home', 'Logout', 'Song', 'Advert', 'Settings', 'Submitdown', \
               'Submitup', 'Thumbsdown', 'Thumbsup', 'Upgrade', 'listen_length', \
               'gender', 'state', 'os', 'browser', 'max_ts', 'min_ts']
    # rename columns to readable format
    df_agg = reduce(lambda df_agg, idx: df_agg.withColumnRenamed(old_columns[idx], new_columns[idx]), range(len(old_columns)), df_agg)
    df_agg = df_agg.withColumn("listen_length_hr", col('listen_length')/(3600.0))
    # number of days member
    df_agg = df_agg.withColumn("member_length_day", (col('max_ts')-col('min_ts'))/(86400.0*1000))
    
    return df_agg

In [144]:
df_agg = make_agg_join_df(df_valid, col_name_new_list)

In [145]:
df_agg.take(6)

[Row(userId='100010', About=1, Friend=4, Playlist=7, Churn=0, Downgrade=0, Error=0, Help=2, Home=11, Logout=5, Song=275, Advert=52, Settings=0, Submitdown=0, Submitup=0, Thumbsdown=5, Thumbsup=17, Upgrade=2, listen_length=66940.89735000003, gender='F', state='CT', os='iOS', browser='Mobile Safari', max_ts=1542823952000, min_ts=1539003534000, listen_length_hr=18.59469370833334, member_length_day=44.21780092592593),
 Row(userId='200002', About=3, Friend=4, Playlist=8, Churn=0, Downgrade=5, Error=0, Help=2, Home=20, Logout=5, Song=387, Advert=7, Settings=3, Submitdown=0, Submitup=1, Thumbsdown=6, Thumbsup=21, Upgrade=2, listen_length=94008.87593999993, gender='M', state='WI', os='iOS', browser='Mobile Safari', max_ts=1542324354000, min_ts=1538393430000, listen_length_hr=26.113576649999978, member_length_day=45.496805555555554),
 Row(userId='125', About=0, Friend=0, Playlist=0, Churn=1, Downgrade=0, Error=0, Help=0, Home=0, Logout=0, Song=8, Advert=1, Settings=0, Submitdown=0, Submitup=0, 

In [95]:
df_agg_3.show()

+------+------+-----+--------+-------------+-------------+
|userId|gender|state|      os|      browser|      max(ts)|
+------+------+-----+--------+-------------+-------------+
|    10|     M|   MS|Mac OS X|       Chrome|1542631788000|
|   100|     M|   TX| Windows|      Firefox|1543587349000|
|100001|     F|   FL|Mac OS X|       Chrome|1538498205000|
|100002|     F|   CA| Windows|      Firefox|1543799476000|
|100003|     F|   FL| Windows|      Firefox|1539274781000|
|100004|     F|   NY| Windows|           IE|1543459065000|
|100005|     M|   LA|Mac OS X|       Chrome|1539971825000|
|100006|     F|   MI| Windows|       Chrome|1538753070000|
|100007|     F|   AR| Windows|       Chrome|1543491909000|
|100008|     F|   CA|Mac OS X|      Firefox|1543335219000|
|100009|     M|   CA| Windows|       Chrome|1540611104000|
|100010|     F|   CT|     iOS|Mobile Safari|1542823952000|
|100011|     M|   OR| Windows|      Firefox|1538417085000|
|100012|     M|   WI|Mac OS X|       Safari|154110090000

In [93]:
df_agg_3.select("state").dropDuplicates().sort("state").count()

39

In [96]:
df_agg_4.show()

+------+-------------+
|userId|      min(ts)|
+------+-------------+
|    10|1538965220000|
|   100|1538492662000|
|100001|1538376504000|
|100002|1539039445000|
|100003|1539100420000|
|100004|1538534835000|
|100005|1538407015000|
|100006|1538747464000|
|100007|1538465403000|
|100008|1539101664000|
|100009|1538374371000|
|100010|1539003534000|
|100011|1538414422000|
|100012|1538586080000|
|100013|1538503066000|
|100014|1539177136000|
|100015|1538430159000|
|100016|1538408566000|
|100017|1540050891000|
|100018|1538743628000|
+------+-------------+
only showing top 20 rows



In [89]:
df_agg_2.count()

225

In [77]:
df_agg_2.show()

+------+------------------+
|userId|       sum(length)|
+------+------------------+
|    10|166866.37251000004|
|   100|  672877.856590001|
|100001|35073.742150000005|
|100002| 49559.91809999999|
|100003|12306.419320000003|
|100004|231480.38928999988|
|100005| 37037.12201000001|
|100006|        5854.89105|
|100007|104495.32492000007|
|100008|191396.79601999995|
|100009|129745.78967999991|
|100010| 66940.89735000003|
|100011|         2841.6995|
|100012|115503.35311000001|
|100013|280391.79410999996|
|100014| 67703.47208000004|
|100015|194617.91655000017|
|100016| 129376.7523399999|
|100017|12261.723399999999|
|100018|248530.32762999993|
+------+------------------+
only showing top 20 rows



In [66]:
from functools import reduce
oldColumns = df_agg.schema.names
print(oldColumns)
df_agg = reduce(lambda df_agg, idx: df_agg.withColumnRenamed(oldColumns[idx], col_name_new_list[idx]), range(len(oldColumns)), df_agg)

['userId', 'sum(About_)', 'sum(Add Friend_)', 'sum(Add to Playlist_)', 'sum(Cancellation Confirmation_)', 'sum(Downgrade_)', 'sum(Error_)', 'sum(Help_)', 'sum(NextSong_)', 'sum(Roll Advert_)', 'sum(Settings_)', 'sum(Thumbs Down_)', 'sum(Thumbs Up_)', 'sum(Upgrade_)']


In [67]:
df_agg.show()

+------+------+-----------+----------------+--------------------------+----------+------+-----+---------+------------+---------+------------+----------+--------+
|userId|About_|Add Friend_|Add to Playlist_|Cancellation Confirmation_|Downgrade_|Error_|Help_|NextSong_|Roll Advert_|Settings_|Thumbs Down_|Thumbs Up_|Upgrade_|
+------+------+-----------+----------------+--------------------------+----------+------+-----+---------+------------+---------+------------+----------+--------+
|    10|     2|         12|               9|                         0|         7|     0|    1|      673|           1|        7|           4|        37|       0|
|   100|    12|         49|              61|                         0|        30|     3|   18|     2682|          25|       11|          27|       148|       1|
|100001|     0|          2|               3|                         1|         0|     1|    1|      133|          14|        1|           2|         8|       2|
|100002|     0|          1| 

In [68]:
df_agg.count()

225

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.